In [1]:
import pandas as pd
import recordlinkage
import pathlib

In [2]:
Data_folder: str = pathlib.Path('C:\BBBrothers\BBB BETTER AND BETTER FILES\Data\merged_file.csv')

In [3]:
record_data = pd.read_csv(Data_folder)

C:\Users\arcoo\AppData\Local\Temp\ipykernel_17236\1740684639.py:1: DtypeWarning: Columns (2,3,4,5,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  record_data = pd.read_csv(Data_folder)


In [4]:
record_data

,ID,firm_id,company_name,condensed_name,main,legal,address_1,address_2,city,state,zip,zip_4,phone,email,url
0,1,2.0,"Able Fence, Inc.",ablefenceinc,True,True,78 Acker St E,NaN,Saint Paul,MN,55117,5507,6512224355,NaN,http://www.albinchapel.com/
1,2,5.0,Albin Chapel,albinchapel,False,False,2200 Nicollet Ave,NaN,Minneapolis,MN,55404,3301,9529149410,office@albinchapel.com,NaN
2,3,5.0,Albin Funeral Chapel Inc,albinfuneralchapelinc,False,False,6855 Rowland Rd,NaN,Eden Prairie,MN,55344,3209,6128711418,jimalbinson@gmail.com,http://www.arthurwilliamsoptical.com/
3,4,5.0,"Albin Endeavor, Inc.",albinendeavorinc,True,True,PO Box 46147,NaN,Eden Prairie,MN,55344,2847,6122700491,NaN,NaN
4,5,7.0,Albrecht Company,albrechtcompany,True,False,2024 Blackberry Ln,NaN,Wayzata,MN,55391,2006,6516334510,mail@albrechtcompany.com,http://www.ablemovers.net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309324,327934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7016621044,NaN,NaN
309325,327935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084001,NaN,NaN
309326,327936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084150,NaN,NaN
309327,327937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6125672270,NaN,NaN


In [5]:
indexer = recordlinkage.Index()
data_sample = record_data.head(100000)
indexer.block("condensed_name")
canidate_links = indexer.index(data_sample)

In [6]:
print(len(data_sample),  len(canidate_links))

100000 14847


In [7]:
compare_cl = recordlinkage.Compare()
compare_cl.string("condensed_name", "condensed_name",method ="jarowinkler",threshold=0.90, label="condensed_name")
compare_cl.string(
    "company_name", "company_name", method="jarowinkler", threshold=0.90, label="company_name"
)
compare_cl.string("address_1", "address_1", method="jarowinkler", threshold=0.85, label="address_1")
compare_cl.exact("state", "state", label="state")
compare_cl.string(
    "email", "email", method="jarowinkler", threshold=0.85, label="email"
)
compare_cl.string(
    "url", "url", label = "url"
)
features = compare_cl.compute(canidate_links, data_sample)

In [8]:
features.head(10)

condensed_name  company_name  address_1  state  email       url
29    27                1.0           1.0        0.0      0    0.0  0.484848
8412  33                1.0           1.0        0.0      1    0.0  0.529412
13063 33                1.0           1.0        0.0      1    0.0  0.400000
      8412              1.0           1.0        0.0      1    0.0  0.342857
13332 33                1.0           1.0        0.0      1    0.0  0.533333
      8412              1.0           1.0        0.0      1    0.0  0.558824
      13063             1.0           1.0        0.0      1    0.0  0.428571
13426 33                1.0           1.0        0.0      1    0.0  0.406250
      8412              1.0           1.0        0.0      1    0.0  0.441176
      13063             1.0           1.0        0.0      1    0.0  0.400000

In [9]:
features.describe()

,condensed_name,company_name,address_1,state,email,url
count,14847.0,14847.000000,14847.000000,14847.000000,14847.000000,14847.000000
mean,1.0,0.971711,0.010777,0.686334,0.001819,0.285389
std,0.0,0.165802,0.103253,0.463998,0.042607,0.243239
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,1.000000,0.000000,1.000000,0.000000,0.350000
75%,1.0,1.000000,0.000000,1.000000,0.000000,0.500000
max,1.0,1.000000,1.000000,1.000000,1.000000,0.964286


In [10]:
round(features.sum(axis=1)).value_counts().sort_index(ascending=False)

5.0      33
4.0    2790
3.0    8282
2.0    3652
1.0      90
dtype: int64

In [11]:
matches = features[round(features.sum(axis=1)) > 4]
matches

condensed_name  company_name  address_1  state  email       url
2670  2669              1.0           0.0        1.0      1    1.0  0.564103
66198 21115             1.0           1.0        1.0      1    0.0  0.555556
44591 43405             1.0           1.0        1.0      1    0.0  0.550000
10113 9404              1.0           1.0        1.0      1    0.0  0.542857
41427 6804              1.0           1.0        1.0      1    0.0  0.592593
71654 41427             1.0           1.0        1.0      1    0.0  0.576923
66037 36212             1.0           1.0        1.0      1    0.0  0.612903
8982  8981              1.0           1.0        1.0      1    0.0  0.629630
12546 8981              1.0           1.0        1.0      1    0.0  0.545455
      8982              1.0           1.0        1.0      1    0.0  0.515152
50598 8981              1.0           1.0        1.0      1    0.0  0.592593
      8982              1.0           1.0        1.0      1    0.0  0.640000
      12546             1.0           1.0        1.0      1    0.0  0.575758
33672 15054             1.0           1.0        1.0      1    0.0  0.620690
93829 60821             1.0           1.0        1.0      1    0.0  0.545455
37721 13018             1.0           1.0        1.0      1    0.0  0.548387
47162 36329             1.0           1.0        1.0      1    0.0  0.520000
32247 9797              1.0           1.0        1.0      1    0.0  0.533333
33666 31788             1.0           1.0        1.0      1    0.0  0.518519
26054 18185             1.0           1.0        1.0      1    0.0  0.531250
34640 18952             1.0           1.0        1.0      1    0.0  0.586207
63346 35255             1.0           1.0        1.0      1    0.0  0.515152
24010 24009             1.0           1.0        0.0      1    1.0  0.548387
24855 24854             1.0           1.0        0.0      1    1.0  0.586207
26427 26426             1.0           1.0        1.0      1    0.0  0.551724
29443 29442             1.0           1.0        1.0      1    0.0  0.566667
29900 29899             1.0           1.0        1.0      1    0.0  0.695652
69163 65242             1.0           1.0        0.0      1    1.0  0.607143
41654 38393             1.0           1.0        1.0      1    0.0  0.548387
41028 41026             1.0           1.0        1.0      1    0.0  0.576923
65400 65399             1.0           1.0        0.0      1    1.0  0.548387
77082 77081             1.0           1.0        1.0      1    0.0  0.531250
98821 98820             1.0           1.0        1.0      1    1.0  0.000000

In [12]:
matches.rename_axis({'ID_1','ID_2'})

,,condensed_name,company_name,address_1,state,email,url
ID_2,ID_1,,,,,,
2670,2669,1.0,0.0,1.0,1,1.0,0.564103
66198,21115,1.0,1.0,1.0,1,0.0,0.555556
44591,43405,1.0,1.0,1.0,1,0.0,0.550000
10113,9404,1.0,1.0,1.0,1,0.0,0.542857
41427,6804,1.0,1.0,1.0,1,0.0,0.592593
71654,41427,1.0,1.0,1.0,1,0.0,0.576923
66037,36212,1.0,1.0,1.0,1,0.0,0.612903
8982,8981,1.0,1.0,1.0,1,0.0,0.629630
12546,8981,1.0,1.0,1.0,1,0.0,0.545455


In [13]:
matches = matches.reset_index(names=['ID1','ID2'])

In [14]:
matches

,ID1,ID2,condensed_name,company_name,address_1,state,email,url
0,2670,2669,1.0,0.0,1.0,1,1.0,0.564103
1,66198,21115,1.0,1.0,1.0,1,0.0,0.555556
2,44591,43405,1.0,1.0,1.0,1,0.0,0.550000
3,10113,9404,1.0,1.0,1.0,1,0.0,0.542857
4,41427,6804,1.0,1.0,1.0,1,0.0,0.592593
5,71654,41427,1.0,1.0,1.0,1,0.0,0.576923
6,66037,36212,1.0,1.0,1.0,1,0.0,0.612903
7,8982,8981,1.0,1.0,1.0,1,0.0,0.629630
8,12546,8981,1.0,1.0,1.0,1,0.0,0.545455
9,12546,8982,1.0,1.0,1.0,1,0.0,0.515152


In [15]:
def getColumnValues(df, column_name: str) -> list[str]:
    return df[column_name]

In [38]:
column_values = getColumnValues(matches, "ID1")
id1 = [record_data.columns.values.tolist()]
for value in column_values:
    list = record_data[record_data['ID'] == value]
    id1.append(list.values.tolist())



[['ID', 'firm_id', 'company_name', 'condensed_name', 'main', 'legal', 'address_1', 'address_2', 'city', 'state', 'zip', 'zip_4', 'phone', 'email', 'url'], [[2670, 2705.0, 'Movers, Inc.', 'moversinc', True, True, 'PO Box 335', '19300 Linden Dr', 'Rogers', 'MN', '55374', '0335', nan, 'joe@kissalaska.com', 'https://www.millsauto.com']], [[66198, 96088638.0, 'Culligan Water, Inc.', 'culliganwaterinc', False, False, '363 5th Ave N', nan, 'Bayport', 'MN', '55003', 1235.0, '6513876401', 'viptowing2013@gmail.com', 'http://www.habitaware.com/']], [[44591, 96017061.0, 'AmericInn Lodge & Suites', 'americinnlodgeandsuites', True, True, '310 N Lakeshore Dr', nan, 'Lake City', 'MN', '55041', '1553.0', '9528887545', 'deluxmotel@gmail.com', nan]], [[10113, 8006385.0, 'BDS Laundry Systems', 'bdslaundrysystems', False, False, 'PO Box 9235', nan, 'Fargo', 'ND', '58106', '9235', '6123332885', 'sewerworks11@yahoo.com', 'https://plus.google.com/u/0/+swansonvitamins/posts']], [[41427, 96013224.0, 'Craig Carl

In [47]:
column_values = getColumnValues(matches, "ID2")
id2 = [record_data.columns.values.tolist()]
for value in column_values:
    list = record_data[record_data['ID'] == value]
    id2.append(list.values.tolist())

In [52]:
flattened_data = [item for sublist in id1[1:] for item in sublist]
df1=pd.DataFrame(flattened_data, columns= ['ID', 'firm_id', 'company_name', 'condensed_name', 'main', 'legal', 'address_1', 'address_2', 'city', 'state', 'zip', 'zip_4', 'phone', 'email', 'url'])
df1.drop(['condensed_name','main','legal','zip','zip_4'],inplace=True,axis=1)
df1

,ID,firm_id,company_name,address_1,address_2,city,state,phone,email,url
0,2670,2705.0,"Movers, Inc.",PO Box 335,19300 Linden Dr,Rogers,MN,NaN,joe@kissalaska.com,https://www.millsauto.com
1,66198,96088638.0,"Culligan Water, Inc.",363 5th Ave N,NaN,Bayport,MN,6513876401,viptowing2013@gmail.com,http://www.habitaware.com/
2,44591,96017061.0,AmericInn Lodge & Suites,310 N Lakeshore Dr,NaN,Lake City,MN,9528887545,deluxmotel@gmail.com,NaN
3,10113,8006385.0,BDS Laundry Systems,PO Box 9235,NaN,Fargo,ND,6123332885,sewerworks11@yahoo.com,https://plus.google.com/u/0/+swansonvitamins/p...
4,41427,96013224.0,Craig Carlson United Painting & Design,12201 Champlin Dr,NaN,Champlin,MN,NaN,lilamaepereze@yahoo.com,http://www.petpalacedoggrooming.com
5,71654,96099246.0,Reyes Enterprises,1960 Cliff Lake Rd STE 119,NaN,Eagan,MN,6517758954,jessica@corehealthbemidji.com,NaN
6,66037,96088309.0,"IGWT Construction & Remodeling, LLC",PO Box 727,NaN,North Branch,MN,9525441035,info@wmlc.biz,http://www.ottersonmentalhealth.com
7,8982,8001827.0,Great American Insurance Co.,303 10th Ave S,NaN,Buffalo,MN,2183261200,tturner@supportcollectors.com,http://www.beckerplasticsurgery.com
8,12546,14000611.0,A C A,709 Voyageur Rd,NaN,Ely,MN,6514511551,townandcountryexc@gmail.com,http://www.stfrancisanimalandbird.com
9,12546,14000611.0,A C A,709 Voyageur Rd,NaN,Ely,MN,6514511551,townandcountryexc@gmail.com,http://www.stfrancisanimalandbird.com


In [49]:
flattened_data2 = [item for sublist in id2[1:] for item in sublist]
df2=pd.DataFrame(flattened_data2, columns= ['ID', 'firm_id', 'company_name', 'condensed_name', 'main', 'legal', 'address_1', 'address_2', 'city', 'state', 'zip', 'zip_4', 'phone', 'email', 'url'])
df2

,ID,firm_id,company_name,condensed_name,main,legal,address_1,address_2,city,state,zip,zip_4,phone,email,url
0,2669,2687.0,Menards Cashway Lumber,menardscashwaylumber,False,False,500 Cardigan Rd,NaN,Saint Paul,MN,55126,3996,9528301133,info@counselwealthmanagement.com,http://www.tcwreckersales.com
1,21115,24001270.0,"House of Wood, Inc",houseofwoodinc,True,True,PO Box 877,NaN,Watertown,MN,55388,0877,6516462700,crogers@cadan.com,http://clarksseamlessgutter.som
2,43405,96015429.0,Endless Masonry,endlessmasonry,False,False,1606 Snowflake Dr,NaN,Eagan,MN,55121,1765.0,7634249951,smile@heartofthecitydental.com,http://www.appleacademychildcare.com
3,9404,8002859.0,Master Construction Co Inc,masterconstructioncoinc,True,True,342 42nd St S,NaN,Fargo,ND,58103,1132,7634202140,a1towing@yahoo.com,http://www.kohnenshvac.rheemdealer.net
4,6804,4000137.0,DDD Motel Corporation,dddmotelcorporation,False,True,1681 University Ave W,NaN,Saint Paul,MN,55104,3726,6126421234,suem@animalwellnessmg.com,http://www.wise-furnitureco.com
5,41427,96013224.0,Craig Carlson United Painting & Design,craigcarlsonunitedpaintinganddesign,True,True,12201 Champlin Dr,NaN,Champlin,MN,55316,1930.0,NaN,lilamaepereze@yahoo.com,http://www.petpalacedoggrooming.com
6,36212,96007539.0,Richard Vikander Remodeling,richardvikanderremodeling,True,False,NaN,NaN,NaN,NaN,NaN,NaN,9522240732,portraits@portraitsfromtheheart.net,http://www.powerliftmn.com
7,8981,8001826.0,Graybar Electric Company Inc,graybarelectriccompanyinc,True,True,2500 Park Ave,NaN,Minneapolis,MN,55404,4403,9523804930,info@summitcollects.com,http://www.basinelectric.com
8,8981,8001826.0,Graybar Electric Company Inc,graybarelectriccompanyinc,True,True,2500 Park Ave,NaN,Minneapolis,MN,55404,4403,9523804930,info@summitcollects.com,http://www.basinelectric.com
9,8982,8001827.0,Great American Insurance Co.,greatamericaninsuranceco,True,True,303 10th Ave S,NaN,Buffalo,MN,55313,2302,2183261200,tturner@supportcollectors.com,http://www.beckerplasticsurgery.com
